In [1]:
pip install tensorflow tensorflow-hub nltk


^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense

def text_encoder(vocab_size, embedding_dim, max_length):
    model = tf.keras.Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        LSTM(256, return_sequences=False),
        Dense(256, activation='relu')
    ])
    return model


In [ ]:
from tensorflow.keras.layers import Conv2DTranspose, Dense, Reshape, BatchNormalization, ReLU

def generator():
    model = tf.keras.Sequential([
        Dense(4 * 4 * 512, use_bias=False, input_shape=(256,)),  # Input: text embedding
        BatchNormalization(),
        ReLU(),
        Reshape((4, 4, 512)),
        Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        BatchNormalization(),
        ReLU(),
        Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        BatchNormalization(),
        ReLU(),
        Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', activation='tanh')  # Output: RGB image
    ])
    return model


In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten, LeakyReLU, Concatenate

def discriminator():
    image_input = tf.keras.Input(shape=(64, 64, 3))
    text_input = tf.keras.Input(shape=(256,))

    x = Conv2D(64, (5, 5), strides=(2, 2), padding='same')(image_input)
    x = LeakyReLU(0.2)(x)
    x = Conv2D(128, (5, 5), strides=(2, 2), padding='same')(x)
    x = LeakyReLU(0.2)(x)
    x = Flatten()(x)

    combined = Concatenate()([x, text_input])
    combined = Dense(1, activation='sigmoid')(combined)

    return tf.keras.Model([image_input, text_input], combined)


In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss


In [ ]:
@tf.function
def train_step(generator, discriminator, text_input, image_input, text_embedding, optimizer_gen, optimizer_disc):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(text_embedding, training=True)
        real_output = discriminator([image_input, text_embedding], training=True)
        fake_output = discriminator([generated_images, text_embedding], training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_disc = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    optimizer_gen.apply_gradients(zip(gradients_gen, generator.trainable_variables))
    optimizer_disc.apply_gradients(zip(gradients_disc, discriminator.trainable_variables))


In [ ]:
def generate_and_save_images(generator, text_embedding, epoch):
    predictions = generator(text_embedding, training=False)
    # Convert to displayable format
    for i in range(predictions.shape[0]):
        plt.imshow((predictions[i] + 1) / 2)
        plt.axis('off')
    plt.show()
